In [1]:
import spacy
from spacy.matcher import Matcher 
from spacy.tokens import Span 
from spacy import displacy
import spacy
import neuralcoref

/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
  return f(*args, **kwds)


In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
nlp.add_pipe(nlp.create_pipe('merge_subtokens'), after = 'parser')
nlp.add_pipe(nlp.create_pipe("merge_entities"), after = 'ner')
neuralcoref.add_to_pipe(nlp)

In [4]:
nlp.pipeline

[('tagger', <spacy.pipeline.pipes.Tagger at 0x7fb801371d50>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x7fb80135a440>),
 ('merge_subtokens',
  <function spacy.pipeline.functions.merge_subtokens(doc, label='subtok')>),
 ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x7fb80135a3d0>),
 ('merge_entities', <function spacy.pipeline.functions.merge_entities(doc)>),
 ('neuralcoref', <neuralcoref.neuralcoref.NeuralCoref at 0x7fb801370b50>)]

Their understanding that x did y

(coref, x, y)

# Assessments

In [5]:
with open('assessments.tff') as f:
    content = f.read()
    print(content)

#class="assessments"
(our|my) (opinion|understanding) (is|was) that
it (is|was) (our|my) (opinion|understanding) (that)?
in (our|my) opinion
(our|my) take on
it (seems|seemed) to (us|me) (that)?
it (seems|seemed) (that)?
it would seem to (us|me)?
it would appear to (us|me)?
it appears to (us|me)?
(the|my|our) ([\w]+[ ])?point is (that)?
(the|my|our) ([\w]+[ \,]*){1,2} point is (that)?
it (looks|looked) to (us|me) (as if|like)
it (looks|looked) (as if|like|that way)
(we|i) (have|get|got) the impression (that)?
(our|my) impression (was|is) (that)?
in (our|my) book
to (our|my) mind
to (our|my) way of thinking
as far as (I am|I was|we are|we were) concerned
if you ask (me|us)
(our|my) feeling (is|was|would be)
from where (I\'m|I am) (standing|sitting)
(we|I) (don\'t)? think (that)?
all (we\'re|I\'m) saying is
what (I\'m|we\'re) saying is
(we\'re|I\'m) (not)? saying that
what (we\'re|i\'m) trying to say is
what (we|i) mean is (that)?


In [6]:
pattern1 = [{'LOWER':'it', 'OP':'?'},
            {'LEMMA': 'be', 'OP':'?'},
            {'DEP' :'poss'},
            {'LOWER': {'IN':['understanding', 'opinion']}},
            {'LEMMA': 'be', 'OP':'?'},
            {'LOWER': 'that', 'OP':'?'},
            {'TEXT': {"REGEX":"."}, 'OP':"+"},
            {'TEXT': {"REGEX": "\."}}
           ]

In [8]:
pattern2 = [{'LOWER':'in'},
            {'DEP' :'poss'},
            {'LOWER': 'opinion'},
            {'TEXT': {"REGEX":"."}, 'OP':"+"},
            {'TEXT': {"REGEX": "\."}}
           ]

In [ ]:
pattern3 = [{'LOWER':'it'},
            {'LOWER': 'would', 'OP':'?'},
            {'LEMMA' :{'IN':['seem', 'appear']}},
            {'LOWER': 'to', 'OP':'?'},
            {'POS': 'PRON', 'OP':'?'},
            {'LOWER': 'that', 'OP':'?'},
            {'TEXT': {"REGEX":"."}, 'OP':"+"},
            {'TEXT': {"REGEX": "(\.|$)"}}
           ]

In [ ]:
pattern4 = [{'DEP':'poss'},
            {'IS_ALPHA': 'True', 'OP':'*'},
            {'LOWER' : 'point'},
            {'LEMMA': 'be'}
           ]

In [ ]:
pattern5 = [{'LOWER':'it'},
            {'LEMMA': 'look'},
            {'LOWER' : 'to'},
            {'POS': 'PRON'}
           ]

In [15]:
doc = nlp('It looked to me as if this was wrong.')

In [16]:
for i in doc:
    print(i, "->", i.pos_)

It -> PRON
looked -> VERB
to -> ADP
me -> PRON
as -> SCONJ
if -> SCONJ
this -> DET
was -> AUX
wrong -> ADJ
. -> PUNCT


In [ ]:
# Whos understanding that who did what

In [7]:
text = "The top Democrats on the House and Senate Foreign Relations Committee on Saturday began a probe into the firing, saying it was their understanding that Secretary of State Mike Pompeo personally recommended Linick’s sacking because the inspector general 'had opened an investigation into wrongdoing by Secretary Pompeo himself.'"

In [92]:
text = "it was my understanding that the dog would not leave the house."

In [187]:
text = "The internal auditors were able to supply ‘reasonable’ assurance in their audit opinion for 2014-15, that, based on the work undertaken and implementation of actions by management, that the College’s systems of internal control, governance and risk management were operating adequately and there were no instances where nay breakdown of control resulted in a material discrepancy. This level of assurance was the highest that could be offered. It was their opinion that the College has adequate and effective management, control and governance processes in place to manage the achievement of its objectives."

In [188]:
text

'The internal auditors were able to supply ‘reasonable’ assurance in their audit opinion for 2014-15, that, based on the work undertaken and implementation of actions by management, that the College’s systems of internal control, governance and risk management were operating adequately and there were no instances where nay breakdown of control resulted in a material discrepancy. This level of assurance was the highest that could be offered. It was their opinion that the College has adequate and effective management, control and governance processes in place to manage the achievement of its objectives.'

In [221]:
text = 'He locked himself in his office and held on to his desk as the ache in his head “gave way to a hum, a low chanting hum, like the one that comes when one is just going under an anesthetic.” Soon he was hot all over and shaking in fear. Later that day, Hayden was admitted to Walter Reed Hospital, where he was subjected to a battery of tests. After he had spent weeks in the hospital, three doctors came to see him. “They seemed to have something on their minds,” he wrote. “With the greatest consideration, calmly and with expressions of regret, I was informed that it was their opinion that I had best get my affairs in shape as I would probably not live for more than a month, or at best would be permanently insane.”'

In [25]:
text = "A few days ago I had occasion to have a conversation with Admiral Lindman, the chief of the Swedish trade commission, who has been a Prime Minister of Sweden and a leader of the conservative party. Admiral Lindman states that he has been given full power by the King in dealing with these matters and that he was [Page 285]responsible for the nomination of the members of his commission. He said that they had entered into these negotiations with the best will possible and with the intention of arriving at some agreement with the British delegates, and while they were willing to make any reasonable concessions, they were not willing to do anything inconsistent with the sovereign rights of the Swedish nation. It was their opinion that it was the purpose of the British Government to induce Sweden to allow the passage in transit of any amount of goods to Russia and at the same time to restrict Sweden to a minimum amount for her own use, and of course not allow the passage of any goods from Sweden to Germany. The Swedish Government has placed a prohibition on practically all goods that might find their way from this country into Germany and yet the English Government was not willing to grant any freedom to Sweden in the matter of its commerce."

In [26]:
doc = nlp(text)

In [27]:
pattern1 = [{'LOWER':'it'},
            {'LEMMA': 'be'},
            {'DEP' :'poss'},
            {"LOWER": {'IN':['understanding', 'opinion']}}, 
            {'LOWER': 'that'},
            {"TEXT": {"REGEX":"."}, 'OP':"+"},
            {"TEXT": {"REGEX": "\."}}
           ]

In [28]:
matcher = Matcher(nlp.vocab)

In [29]:
matcher.add("assessments", None, pattern1)

In [30]:
matches = matcher(doc)

In [31]:
span = doc[matches[0][1]:matches[0][2]]
print(span.sent)

It was their opinion that it was the purpose of the British Government to induce Sweden to allow the passage in transit of any amount of goods to Russia and at the same time to restrict Sweden to a minimum amount for her own use, and of course not allow the passage of any goods from Sweden to Germany.


[the British Government, Sweden, Russia, Sweden, Sweden, Germany]

In [23]:
matches

[]

In [244]:
from spacy import displacy

In [245]:
displacy.render(span, style="dep")

In [251]:
span[]

It was their opinion that it was the purpose of the British Government to induce Sweden to allow the passage in transit of any amount of goods to Russia and at the same time to restrict Sweden to a minimum amount for her own use, and of course not allow the passage of any goods from Sweden to Germany.

# Who's opinion/understanding?

In [242]:
for z in span:
    if z.text.lower() in ['understanding', 'opinion']:
        for x in z.children:
            if x.dep_ == 'poss':
                if x._.coref_clusters:
                    print(x._.coref_clusters[0].mentions[0])
                else:
                    print(x)

they


# That Who or What?

In [249]:
for z in span:
    if z.text == 'that':
        for x in z.head.children:
            if x.dep_ in ['nsubj', 'nsubjpass']:
                for b in x.subtree:
                    print(b)

it


# Did what?

In [250]:
for z in span:
    if z.text == 'that':
        for x in z.head.lefts:
            if x.dep_ in ['aux', 'auxpass', 'neg']:
                print(x)
            
        print(z.head)
            
        for x in z.head.rights:
            if x.dep_ == 'dobj':
                for a in x.subtree:
                    print(a)

was


In [5]:
# http://markneumann.xyz/blog/dependency_matcher/

In [35]:
Span.set_extension("IS_OPINION", default=None)

In [45]:
class Opinion_Detector(object):
    
    def __init__(self, nlp):
        
        Span.set_extension("IS_ASSESSMENT", default=False, force = True)
        self.matcher = Matcher(nlp.vocab)
        self.matcher.add("Opinion", None,
            
            [{'LOWER':'it'},
            {'LEMMA': 'be'},
            {'DEP' :'poss'},
            {"LOWER": {'IN':['understanding', 'opinion']}}, 
            {'LOWER': 'that'}],
                         
            [{'LOWER':'it'},
            {'LOWER': 'would', 'OP':'?'},
            {'LEMMA' :{'IN':['seem', 'appear']}},
            {'LOWER': 'to', 'OP':'?'},
            {'POS': 'PRON', 'OP':'?'},
            {'LOWER': 'that', 'OP':'?'}]
                         
            [],
                         
            [],
                         
                         
            [],
                         
            
            [],
        )
    
    def __call__(self, doc):
        
        # This method is invoked when the component is called on a Doc
        matches = self.matcher(doc)
        for match_id, start, end in matches:
            span = doc[start:end]
            span.sent._.IS_OPINION = True
        return doc

In [66]:
nlp = spacy.load("en_core_web_sm")
op_tag = Opinion_Sentence(nlp)
nlp.add_pipe(op_tag, last=True)  # Add component to the pipeline
text = "Every day they tried the same thing. Sometimes it worked. It seemed to us that it would be difficult. It was their understanding that the rich just got richer."
doc = nlp(text)

In [67]:
[i for i in doc.sents if i._.IS_OPINION == True]

[It seemed to us that it would be difficult.,
 It was their understanding that the rich just got richer.]

In [ ]:
for token in doc:
    print(token.text, token._.bad_html)